# T038 ·蛋白质配体相互作用预测
* * 注意：** 这篇谈话文章是TeachOpenCADD的一部分，该平台旨在教授特定领域的技能并提供管道模板作为研究项目的起点。

作者：
- Roman Joeres，2022年，[UdS和HIPS药物生物信息学主席](https：//www.helmholtz-hips.de/de/forschung/team/team/spokstoffbioinformatik/），[NextAID]（https：//nextaid.cs.uni-saarland.de/)项目，萨尔大学

## 本期脱口秀的目标

这篇谈话文章的目标是向读者介绍使用图神经网络（GNN）预测蛋白质-配体相互作用的领域。GNN对于将蛋白质和化学分子（配体）等结构数据表示为深度学习模型特别有用。在本期谈话中，我们将展示如何训练深度学习模型来预测蛋白质和配体之间的相互作用。

# * 理论 * 内容
* 蛋白质-配体相互作用预测的相关性
* 工作流
* 生物背景-蛋白质作为图表
* 技术背景
* 图同质网络
* 二元交叉熵损失

# * 实用 * 中的内容
* 计算图形表示
* 图形配体
* 蛋白质到图表
* 数据存储器
* 数据点
* 数据集
* 数据模块
* 网络
* GNN编码器
* 完整模型
* 训练例程

# 参考资料
* 理论背景
* 图形神经网络：
《基于图卷积网络的半监督分类》，[2017年](https://arxiv.org/abs/1609.02907)
《几何深度学习：超越欧几里得数据》，《IEEE信号处理杂志》(2017)，4<b>，18-42](https://doi.org/10.1109/MSP.2017.2693418)
* 基于GNN的蛋白质-配体相互作用预测：
Oztürk等人：“DeepDTA：深度药物-靶向结合亲和力预测”，[生物信息学](2018)，34<b>，i821-i829](https://doi.org/10.1093/bioinformatics/bty593)
阮氏等人Al.：“GraphDTA：用图形神经网络预测药物-靶标结合亲和力”，[生物信息学](2021)，37<b>，1140-1147](https://doi.org/10.1093/bioinformatics/btaa921)
* 图形同构网络：
徐等：《图神经网络有多强大？》，[Arxiv</i>(2018)](https://arxiv.org/abs/1810.00826)
* 实践背景
* [火炬](https://pytorch.org/)
* [火炬Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
* [RDKit](http://rdkit.org/)：Greg Landrum，*RDKit文档*，[PDF](https://www.rdkit.org/UGM/2012/Landrum_RDKit_UGM.Fingerprints.Final.pptx.pdf)，于2019.09.1发布。

## 理论

这部脱口秀结合了您在其他脱口秀中看到的几个主题。在这里，我们将描述如何预测蛋白质和配体之间相互作用的总体想法。如果工作流程中使用的某些技术已经在其他地方提供，我将链接到此。否则，我会在下面解释新事物。

# 蛋白质-配体相互作用预测的相关性
蛋白质-配体的相互作用在研究中很有兴趣，原因有很多，可以在T016中看到。药物发现是蛋白质与配体相互作用预测应用的重要领域之一。在药物发现中，人们想要找到一种针对特定蛋白质的新药。计算机辅助的相互作用预测有助于虚拟筛选过程，在虚拟筛选过程中，许多可能的配体被测试是否与特定的目标蛋白质相互作用。传统上，筛选目标蛋白的潜在药物是在实验室进行的，在实验室中对候选药物进行手动测试，并根据它们的结合亲和力进行排序。结合亲和力是衡量两个分子之间相互作用有多强的指标。结合亲和力越高，相互作用越强，两个分子之间的结合越好。

但人工调查候选人既耗时又昂贵。预测计算机中的绑定事件要快得多，成本也低得多。在这篇演讲中，我们将集中在定性水平上预测蛋白质和配体之间的结合事件，也就是说，如果蛋白质和配体相互结合，亲和力目前并不重要。

# 模型架构

我们训练的输入是一个包含一组蛋白质和一组配体的数据集，以及一个包含每对蛋白质和配体的结合信息的表格。我们将执行监督学习(如__TalktureT022__)，因此，我们将交互列表划分为训练集、验证集和测试集。如上所述，我们将对相互作用进行二进制分类，即一对蛋白质和分子是否相互作用？

我们的网络体系结构的最后一个组件是一个简单的多层感知器(MLP)，如T022中所述。另外两个组件是图神经网络(GNN)，用于从每对数据集中的蛋白质和配体中提取特征。如T035中所讨论的，GNN用于计算保存有关结构的信息的图形结构数据的表示。这些表示被连接成一个向量，该向量用作最终MLP的输入。

！[基本结构](./Images/Basic_Structure_nn.png)
* 图1：*
模型在此笔记本中的可视化。所示的示例性结构取自ID为[4O75]的PDB条目(https://www.rcsb.org/structure/4O75)(关于PDB的介绍，参见说明书T008)。

# 生物背景--蛋白质图表

在这里，我们将关注蛋白质到图形的转换，因为微笑到图形的转换在__T033__中有解释。

在科学中，通常有两种方式来表示蛋白质。通过它们的氨基酸序列或作为__说明书T008_中介绍的PDB结构。由于氨基酸序列不包含结构信息，我们使用蛋白质的PDB文件作为基于结构的模型的输入。在蛋白质的图形表示中，图形的每个节点代表蛋白质中的一个氨基酸。如果两个代表的氨基酸在一定距离内，则画出图中节点之间的边。这相当于蛋白质中两种氨基酸之间的相互作用。为了计算两个氨基酸的距离，我们查看PDB文件中氨基酸的$C\α$原子的坐标。如果两个$C\α原子之间的距离低于一定的距离阈值，我们认为氨基酸相互作用，并在蛋白质的图形表示中插入一条边。这可以在图2中看到。氨基酸中的原子被列举出来。因此，氨基酸的C_α原子是每个氨基酸中特定的碳原子，也存在于蛋白质的骨架中。示例性氨基酸中的$C\α$原子的例子如图2和图3所示。

！[Prot2Graph](./images/prot_graph_creation.png)
* 图2：*
将蛋白质结构的过程和想法可视化为图形。对于这个例子，我们只考虑半胱氨酸的$C\α$原子在7埃的距离阈值内。由于两个半胱氨酸在空间上很接近，它们的硫酸盐生成了一个二硫酸盐桥，并稳定了蛋白质的三维结构，这是我们希望在图形表示中具有的相互作用类型。

！[CAlphas](./Images/calphas.jpg)
* 图3：*
三种典型氨基酸中碳原子的可视化。另外，原子的其他数字I显示了n个氨基酸，但对我们来说，只有$C_\Alpha$原子是有趣的（[来源](https：//chemistry.stackexchange.com/questions/134409/what-exact-makes-a-carbon-atom-%CE%B1-in-a-Protein-residue）)。

# 技术背景

在本节中，我们将重点讨论所提出的解决方案的计算机科学方面。主要讨论具体的GNN架构以及我们使用的节点功能。为了简单起见（并且因为它工作得很好），我们将使用相同的网络架构来计算kinase及其配体的嵌入。

# 图同构网络

有一大堆GNN架构被提出来解决许多问题。如果您想了解最流行的体系结构，可以查看[在PyTorch-Geometric](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers).中实现的卷积层列表在本讲义中，我们将使用GINConv层作为OUT GNN的主干，因为它们已被证明在嵌入分子数据方面功能强大，但其功能仍然易于理解。基于邻居计算节点嵌入的公式为
$$\mathbf{x}^{\prime}_i=h_{\mathbf{\theta}}\left((1+\epsilon)\cdot\mathbf{x}_i+\sum_{j\in\mathcal{N}(i)}\mathbf{x}_j\right)$$
其中，$\mathcal{N}(I)$是节点$i$的邻域集合，$\epsilon$是常量超参数，$h_{\mathbf{\theta}}$是神经网络，如_T022__所示。其思想是将所有邻居嵌入与自己当前的嵌入聚合在一起，并将其放入神经网络中，以提取关于节点及其邻域的信息。

由此可见，GINConv层在其计算中不使用边缘信息。因此，当我们将蛋白质和配体变成图形时，我们唯一需要提取的就是边缘的特征。在这篇演讲教程中，我们将使用一个非常简单的功能，每个节点只包含关于它所代表的氨基酸类型或原子类型的分类信息。关于分类数据的一次热编码的信息在__TalktureT021__中介绍。

GNN模块的最后一个元素是Pooling函数，它用于根据最后一层中的节点嵌入来计算图嵌入。为简单起见(而且因为它的功能出人意料地强大)，我们使用Mean Pooling！这意味着，我们只取最终GINConv层中所有节点嵌入的平均向量。

# 二进制交叉熵损失(BCE损失)

说明书T022__介绍了两个损耗函数，即均方误差和最小均方误差。这两种方法都适用于回归模型的训练，但不适用于分类。对于分类，存在广泛的损失函数，我们将使用[二进制交叉熵Loss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html).

计算损失的公式是
$$-\left[y\cdot\log(x)+(1-y)\cdot\log(1-x)\right]$$
其中，$x$是一个样本的模型输出，$y$是该样本的标签。

其思想是，对于公式，恰好有一项$y$和$1-y$等于$1$，对于正样本，简化为$\log x$，对于负样本，$\log(1-x)$。通过此设置，BCE公式确保在负样本($y=0$)中将预测值$x$推向0，在正样本($y=1$)中将预测值推向$1$。

对于我们的例子，阳性样本($y=1$)是结合的激酶和配体对，那么$x$应该接近于1。因此，在我们的例子中的阴性样本($y=0$)是非结合的激酶和配体对。请注意公式中的前导“-”，这会将公式的其余部分从最大化问题转换为最小化问题。

## 实用

在本实践部分中，我们将讨论实施上述蛋白质-配体相互作用预测解决方案的每一步。我们将从所有所需的导入和一些路径定义开始。

In [ ]:
import math
import random
import os
from pathlib import Path

from rdkit import Chem
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam

In [ ]:
import sys
if sys.platform.startswith(("linux", "darwin")):
    !mamba install -q -y -c pyg pyg

In [ ]:
from torch_geometric.nn import global_mean_pool, GINConv
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

from utils import kiba_preprocessing

In [3]:
HERE = Path("./")
DATA = HERE / "data"
IMGS = HERE / "images"

#该方法调用一个数据预处理管道，该管道技术含量很高，对于这个谈话节目来说没有更大的兴趣。
#该方法基本上将KiBA数据集从Excel表转换为我们需要的格式的结构数据集。
kiba_preprocessing(DATA / "KIBA.csv", DATA / "resources")

KiBA originally contains 52498 ligands and 468 proteins.
KiBA after dropping sparse rows contains 79 ligands and 468 proteins.
KiBA finally contains 79 ligands and 373 proteins.
Preprocessing ligands
After ligand availability analysis KiBA contains 76 ligands and 373 proteins.
Preprocessing ligands finished
Preprocessing proteins
After protein availability analysis KiBA contains 76 ligands and 275 proteins.
Preprocessing proteins finished
Preprocessing interactions
Finally, KiBA comprises 20475 interactions.
Preprocessing interactions finished


# 计算图形表示

# 图形配体

首先，我们将实现配体到图形的转换。对于以下解释，配体具有$N$原子。为了对图进行编码，我们必须计算节点特征的矩阵（$N\乘F$-矩阵，其中$F$是每个节点的特征数量）和由参与节点id对给出的边矩阵。

由于一些与PyTorch几何相关的实现细节，边缘矩阵的格式必须为$2\乘N$。

In [ ]:
#对于我们考虑的每个原子类型，将符号映射到一个数值以进行一次性编码。
atoms_to_num = dict(
    (atom, i) for i, atom in enumerate(["C", "N", "O", "F", "P", "S", "Cl", "Br", "I"])
)


def atom_to_onehot(atom):
    """
    Return the one-hot encoding for an atom given its index in the atoms_to_num dict.

    Parameters
    ----------
    atom: str
        Atomic symbol of the atom to represent

    Returns
    -------
    torch.Tensor
        A one-hot tensor encoding the atoms features.
    """
    #初始化0-vector.
    one_hot = torch.zeros(len(atoms_to_num) + 1, dtype=torch.float)
    # ...并将相应字段设置为1，.
    if atom in atoms_to_num:
        one_hot[atoms_to_num[atom]] = 1.0
    # ...最后一个字段用于表示在一热载体中没有自己字段的原子类型
    else:
        one_hot[len(atoms_to_num)] = 1.0
    return one_hot


def smiles_to_graph(smiles):
    """
    Convert a molecule given as SDF file into a graph.

    Arguments
    ---------
    smiles: str
        Path to the file storing the structural information of the ligand

    Returns
    -------
    Tuple[torch.Tensor, torch.Tensor]
        A pair of node features and edges in the PyTorch Geometric format
    """
    #从SDF文件中读取分子
    mol = Chem.MolFromSmiles(smiles)
    atoms, bonds = [], []
    #检查分子是否有效
    if mol is None:
        print(smiles)
        return None, None

    #搜索所有原子，计算特征向量并将其存储在torch中。张量对象
    for atom in mol.GetAtoms():
        atoms.append(atom_to_onehot(atom.GetSymbol()))
    atoms = torch.stack(atoms)

    #重写分子中的所有键，并将它们以PyTorch Geographic特定格式存储在火炬中。张量，
    for bond in mol.GetBonds():
        bonds.append((bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()))
        bonds.append((bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()))
    bonds = torch.tensor(bonds, dtype=torch.long).T

    return atoms, bonds

# 蛋白质到图表

与我们将配体转换为图形的方式类似，我们将蛋白质转换为图形。输出将是相同的，是一对节点特征和边。要获取有关TSB格式的更多信息，请阅读[This](https：//www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html)。

In [ ]:
#生成从氨基酸到数字的映射以进行一热编码
aa_to_num = dict(
    (aa, i)
    for i, aa in enumerate(
        [
            "ALA",
            "ARG",
            "ASN",
            "ASP",
            "CYS",
            "GLU",
            "GLN",
            "GLY",
            "HIS",
            "ILE",
            "LEU",
            "LYS",
            "MET",
            "PHE",
            "PRO",
            "SER",
            "THR",
            "TRP",
            "TYR",
            "VAL",
            "UNK",
        ]
    )
)


def aa_to_onehot(aa):
    """
    Compute the one-hot vector for an amino acid representing node.

    Arguments
    ---------
    aa: str
        The three-letter code of the amino acid to be represented

    Returns
    -------
    torch.Tensor
        A one-hot tensor encoding the atoms features.
    """
    one_hot = torch.zeros(len(aa_to_num), dtype=torch.float)
    one_hot[aa_to_num[aa]] = 1.0
    return one_hot


def pdb_to_graph(pdb_file_path, max_dist=7.0):
    """
    Extract a graph representation of a protein from the PDB file.

    Arguments
    ---------
    pdb_file_path: str
        Filepath of the PDB file containing structural information on the protein
    max_dist: float
        Distance threshold to apply when computing edges between amino acids

    Returns
    -------
    Tuple[torch.Tensor, torch.Tensor]
        A pair of node features and edges in the PyTorch Geometric format
    """
    #通过寻找CalAlpha原子来读取DBC文件，并根据DBC文件中的位置提取其氨基酸和坐标
    residues = []
    with open(pdb_file_path, "r") as protein:
        for line in protein:
            if line.startswith("ATOM") and line[12:16].strip() == "CA":
                residues.append(
                    (
                        line[17:20].strip(),
                        float(line[30:38].strip()),
                        float(line[38:46].strip()),
                        float(line[46:54].strip()),
                    )
                )
    #最后根据蛋白质中的氨基酸计算节点特征
    node_feat = torch.stack([aa_to_onehot(res[0]) for res in residues])

    #通过迭代所有氨基酸对并计算它们的距离来计算蛋白质的边缘
    edges = []
    for i in range(len(residues)):
        res = residues[i]
        for j in range(i + 1, len(residues)):
            tmp = residues[j]
            if math.dist(res[1:4], tmp[1:4]) <= max_dist:
                edges.append((i, j))
                edges.append((j, i))

    #以PyTorch Geographic格式存储边
    edges = torch.tensor(edges, dtype=torch.long).T

    return node_feat, edges

# 数据存储

在蛋白质-配体相互作用预测中为我们的神经网络存储和表示输入数据与其他神经网络有点不同。因此，我们必须定义自己的类来表示数据。与__Talkorial T008__中训练MLP的主要区别除了图形作为输入之外，还在于我们有两个数据点作为输入。蛋白质的图表和配体的图表。因此，我们需要实施自己的数据基础设施。

# 数据点

通常[PyTorch Geographic](https：//pytorch-geographic.readthedocs.io/en/latest/）的内置[Data]（https：//pytorch-geographic.readthedocs.io/en/latest/)类仅用于表示一个图形，对于我们的任务，数据包含两个图形，因此，我们需要调整功能来计算一个数据点的节点和边的数量。

In [ ]:
class DTIDataPair(Data):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def num_nodes(self):
        return self["lig_x"].size(0) + self["prot_x"].size(0)

    @property
    def num_edges(self):
        return self["lig_edge_index"].size(1) + self["prot_edge_index"].size(1)

    def __inc__(self, key, value, *args, **kwargs):
        """
        Method that is necessary to overwrite for successful batching of DTIDataPair object.
        In case of interest, one can look at this explanation:
        https://pytorch-geometric.readthedocs.io/en/latest/notes/batching.html

        When multiple samples are sent through a network at once, they are aggregated into batches.
        In PyTorch Geometric this is done by copying all n graphs for one batch into one graph with
        n connected components. Because of this, the node ids in the edge_index objects have to be
        changed. As they have to be increased by a fixed offset based on the number of nodes in the
        batch so far, this method computes this offset in case the edge_indices of either the
        proteins or ligands.

        Arguments
        ---------
        key: str
            String name of the field of this class to increment while batching

        Returns
        -------
        torch.Tensor
            A one-element tensor describing how to modify the values when batching.
        """
        if not key.endswith("edge_index"):
            return super().__inc__(key, value, *args, **kwargs)
        lenedg = len("edge_index")
        prefix = key[:-lenedg]
        return self[prefix + "x"].size(0)

# 数据集

这就是真正的数据奇迹发生的地方。在数据集中，我们读取数据点并将其处理为我们想要的图形表示。

In [ ]:
class DTIDataset(InMemoryDataset):
    def __init__(self, folder_name, file_index):
        self.folder_name = folder_name
        super().__init__(root=folder_name)
        self.data, self.slices = torch.load(self.processed_paths[file_index])

    @property
    def processed_file_names(self):
        """
        Just store the names of the files where the training split, validation split, and test split are stored.

        Returns
        -------
        List[str]
            A list of filenames where the preprocessed data is stored to not recompute the preprocessing every time.
        """
        return ["train.pt", "val.pt", "test.pt"]

    def process(self):
        """
        This function is called internally in the preprocessing routine of PyTorch Geometric and defined how the dataset of PDB files, ligands, and an interaction table is converted into a dataset of graphs, ready for deep learning.
        """
        #计算所有配体图并将它们存储为字典，其中名称为键，图形为值
        ligand_graphs = dict()
        with open(Path(self.folder_name) / "tables" / "ligands.tsv", "r") as data:
            for line in data.readlines()[1:]:
                chembl_id, smiles = line.strip().split("\t")[:2]
                ligand_graphs[chembl_id] = smiles_to_graph(smiles)

        #计算所有蛋白质图并将它们存储为字典，其中名称为键，图形为值
        protein_graphs = dict(
            [
                (filename[:-4], pdb_to_graph(Path(self.folder_name) / "proteins" / filename))
                for filename in os.listdir(Path(self.folder_name) / "proteins")
            ]
        )

        with open(Path(self.folder_name) / "tables" / "inter.tsv") as inter:
            data_list = []
            for line in inter.readlines()[1:]:
                #阅读带有一个交互示例的行。提取配体和蛋白质ID并从上面的字典中获取它们的图表
                protein, ligand, y = line.strip().split("\t")
                lig_node_feat, lig_edge_index = ligand_graphs[ligand]
                prot_node_feat, prot_edge_index = protein_graphs[protein]

                #如果配体或蛋白质是无效的图表，请跳过此样本.
                if lig_node_feat is None or prot_node_feat is None:
                    print(line.strip())
                    continue

                # ...否则，使用上面的类创建数据点
                data_list.append(
                    DTIDataPair(
                        lig_x=lig_node_feat,
                        lig_edge_index=lig_edge_index,
                        prot_x=prot_node_feat,
                        prot_edge_index=prot_edge_index,
                        y=torch.tensor(float(y), dtype=torch.float),
                    )
                )

            #洗牌数据，并计算有多少个样本进入哪个拆分
            random.shuffle(data_list)
            train_frac = int(len(data_list) * 0.7)
            test_frac = int(len(data_list) * 0.1)

            #然后拆分数据并将其存储以供以后重复使用，而无需运行预处理管道
            train_data, train_slices = self.collate(data_list[:train_frac])
            torch.save((train_data, train_slices), self.processed_paths[0])
            val_data, val_slices = self.collate(data_list[train_frac:-test_frac])
            torch.save((val_data, val_slices), self.processed_paths[1])
            test_data, test_slices = self.collate(data_list[-test_frac:])
            torch.save((test_data, test_slices), self.processed_paths[2])

# 数据模块

这只是一个方便的类，包含数据集的所有三个拆分，并为训练、验证和测试集提供数据加载器。

In [ ]:
class DTIDataModule:
    def __init__(self, folder_name):
        self.train = DTIDataset(folder_name, 0)
        self.val = DTIDataset(folder_name, 1)
        self.test = DTIDataset(folder_name, 2)

    def train_dataloader(self):
        """
        Create and return a dataloader for the training dataset.

        Returns
        -------
        torch_geometric.loaders.DataLoader
            Dataloader on the training dataset
        """
        return DataLoader(
            self.train, batch_size=64, shuffle=True, follow_batch=["prot_x", "lig_x"]
        )

    def val_dataloader(self):
        """
        Create and return a dataloader for the validation dataset.

        Returns
        -------
        torch_geometric.loaders.DataLoader
            Dataloader on the validation dataset
        """
        return DataLoader(self.val, batch_size=64, shuffle=True, follow_batch=["prot_x", "lig_x"])

    def test_dataloader(self):
        """
        Create and return a dataloader for the test dataset.

        Returns
        -------
        torch_geometric.loaders.DataLoader
            Dataloader on the test dataset
        """
        return DataLoader(self.test, batch_size=64, shuffle=True, follow_batch=["prot_x", "lig_x"])

# 网络

在这里，我们将实现理论部分定义的网络。

# GNN编码器

首先，GNN编码器，我们将使用它来嵌入蛋白质和嵌入配体。

In [ ]:
class Encoding(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=64, num_layers=3):
        """
        Encoding to embed structural data using a stack of GINConv layers.

        Arguments
        ---------
        input_dim: int
            Size of the feature vector of the data
        hidden_dim: int
            Number of hidden neurons to use when computing the embeddings
        output_dim: int
            Size of the output vector of the final graph embedding after a final mean pooling
        num_layers: int
            Number of layers to use when computing embedding. This includes input and output layers, so values below 3 are meaningless.
        """
        super().__init__()
        self.layers = (
            [
                #定义输入层
                GINConv(
                    nn.Sequential(
                        nn.Linear(input_dim, hidden_dim),
                        nn.PReLU(),
                        nn.Linear(hidden_dim, hidden_dim),
                        nn.BatchNorm1d(hidden_dim),
                    )
                )
            ]
            + [
                #定义多个隐藏层
                GINConv(
                    nn.Sequential(
                        nn.Linear(hidden_dim, hidden_dim),
                        nn.PReLU(),
                        nn.Linear(hidden_dim, hidden_dim),
                        nn.BatchNorm1d(hidden_dim),
                    )
                )
                for _ in range(num_layers - 2)
            ]
            + [
                #定义输出层
                GINConv(
                    nn.Sequential(
                        nn.Linear(hidden_dim, hidden_dim),
                        nn.PReLU(),
                        nn.Linear(hidden_dim, output_dim),
                        nn.BatchNorm1d(output_dim),
                    )
                )
            ]
        )

    def forward(self, x, edge_index, batch):
        """
        Forward a batch of samples through this network to compute the forward pass.

        Arguments
        ---------
        x: torch.Tensor
            feature matrices of the graphs forwarded through the network
        edge_index: torch.Tensor
            edge indices of the graphs forwarded through the network
        batch: torch.Tensor
            Some internally used information, not relevant for the topic of this talktorial
        """
        for layer in self.layers:
            x = layer(x=x, edge_index=edge_index)
        pool = global_mean_pool(x, batch)
        return F.normalize(pool, dim=1)

# 完整模型

根据理论部分提出的工作流程定义完整模型。

In [ ]:
class DTINetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()

        #为蛋白质和配体创建编码器
        self.prot_encoder = Encoding(21)
        self.lig_encoder = Encoding(10)

        #定义一个简单的FNN来计算最终预测（绑定或不绑定）
        self.combine = torch.nn.Sequential(
            torch.nn.Linear(128, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(256, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(64, 1),
            torch.nn.Sigmoid(),
        )

    def forward(self, data):
        """
        Define the standard forward process of this network.

        Arguments
        ---------
        data: DTIDataPairBatch
            A batch of DTIDataPair samples to be predicted to train on them

        Returns
        -------
        Prediction values for all pairs in the input batch
        """
        #使用蛋白质嵌入器对批次的蛋白质数据计算蛋白质嵌入
        prot_embed = self.prot_encoder(
            x=data.prot_x,
            edge_index=data.prot_edge_index,
            batch=data.prot_x_batch,
        )

        #使用配体嵌入器根据批次的配体数据计算配体嵌入
        lig_embed = self.lig_encoder(
            x=data.lig_x,
            edge_index=data.lig_edge_index,
            batch=data.lig_x_batch,
        )

        #连接两个嵌入并返回FNN的输出
        combined = torch.cat((prot_embed, lig_embed), dim=1)
        return self.combine(combined)

# 训练例行公事

在培训中，我们将使用[Adam优化器](https：//pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam）（这是标准选择)。如上所述，我们使用BCE损失函数来计算模型的预测有多远。该设置的一个特殊之处是我们只训练一个时代。这是因为只有在第一个时期，模型才显示出改进。之后，模型学习了数据集，并没有多大改进。但请随时测试更多时代。平均而言，一个纪元大约需要10分钟才能完成。

In [ ]:
def train(num_epochs=1):
    """
    Implementation of the actual training routine.

    Arguments
    ---------
    num_epochs: int
        Number of epochs to train the model
    """
    #加载数据、模型并定义损失函数
    dataset = DTIDataModule(DATA / "resources")
    model = DTINetwork()
    loss_fn = torch.nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=0.0001)
    epoch_train_acc, epoch_train_loss, epoch_val_acc, epoch_val_loss = [], [], [], []

    #为num_epochs训练
    for e in range(num_epochs):
        print(f"Epoch {e + 1}/{num_epochs}")

        #进行实际培训
        train_loader = dataset.train_dataloader()
        for b, data in enumerate(train_loader):
            #计算模型预测和损失
            pred = model.forward(data).squeeze()
            loss = loss_fn(pred, data.y.squeeze())

            #执行一步反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #报告训练批次的一些统计数据
            pred = pred > 0.5
            epoch_train_acc.append(sum(pred == data.y) / len(pred))
            epoch_train_loss.append(loss.item())
            print(
                f"\rTraining step {(b + 1)}/{len(train_loader)}: Loss: {epoch_train_loss[-1]:.5f}\tAcc: {epoch_train_acc[-1]:.5f}",
                end="",
            )

        torch.save(model.state_dict(), DATA / f"model_{e + 1}.pth")

        #执行最后一个训练时期的验证
        val_loader = dataset.val_dataloader()
        for b, data in enumerate(val_loader):
            #计算模型预测和损失
            pred = model.forward(data).squeeze()
            loss = loss_fn(pred, data.y.squeeze())

            #报告验证批次的一些统计数据
            pred = pred > 0.5
            epoch_val_acc.append(sum(pred == data.y) / len(pred))
            epoch_val_loss.append(loss.item())
            print(
                f"\rValidation step {(b + 1)}/{len(val_loader)}: Loss: {epoch_val_loss[-1]:.5f}\tAcc: {epoch_val_acc[-1]:.5f}",
                end="",
            )

    #测试最终模型
    print()
    test_loss, test_acc = [], []
    test_loader = dataset.test_dataloader()
    for b, data in enumerate(test_loader):
        #计算模型预测和损失
        pred = model.forward(data).squeeze()
        loss = loss_fn(pred, data.y.squeeze())

        #报告验证批次的一些统计数据
        pred = pred > 0.5
        test_acc.append(sum(pred == data.y) / len(pred))
        test_loss.append(loss.item())
        print(f"\rTesting Loss: {test_loss[-1]:.5f}\tAcc: {test_acc[-1]:.5f}", end="")
    print(
        f"\rTesting: Loss: {(sum(test_loss) / len(test_loss)):.5f}\tAcc: {(sum(test_acc) / len(test_acc)):.5f}"
    )

    fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

    ax[0].plot(epoch_train_loss, c="b", label="Train")
    ax[0].plot(
        len(epoch_train_loss) - 1, sum(epoch_val_loss) / len(epoch_val_loss), "rx", label="Val"
    )
    ax[0].set_ylim([0, 1])
    ax[0].set(xlabel="Batches")
    ax[0].set_title("Loss")

    ax[1].plot(epoch_train_acc, c="b", label="Train")
    ax[1].plot(
        len(epoch_train_loss) - 1, sum(epoch_val_loss) / len(epoch_val_loss), "rx", label="Val"
    )
    ax[1].set_ylim([0, 1])
    ax[1].set(xlabel="Batches")
    ax[1].set_title("Accuracy")

    plt.legend()
    plt.savefig(IMGS / "train_perf.png")
    plt.clf()

    return model

In [ ]:
torch.manual_seed(42)
model = train(1)
torch.save(model.state_dict(), DATA / "final_model.pth")

! [火车图表](./ images/train_perf.png)
* 图3：*
第一阶段训练结果的可视化。

## 讨论

如图3所示，损失略有下降，而准确性却停滞不前。蛋白质-配体相互作用预测是一个高度相关且非常复杂的领域。由于蛋白质和配体之间结合的复杂性，例如，配体的哪些原子与蛋白质的哪个位点结合，很难训练一个简单的模型来预测这些相互作用。在这篇脱口秀中，我们讨论了概念证明，该证明将在这篇脱口秀开始时的相关文献中进一步研究。

## 测验

通过这个测验，您可以测试您是否理解了这篇脱口秀的重要教训。

1.为什么我们使用结构数据而不是氨基酸序列和SMILES字符串？
2.我们如何将蛋白质转化为图表？我们为此使用的蛋白质的重要部分是什么？
3.困难：为什么我们需要实现自己的类来表示数据点？